In [1]:
import import_ipynb
import helpers

from create_geojson_methods import create_and_get_gdf
from create_geojson_methods import get_initial_map_data
from create_geojson_methods import clean_gdf_columns
from create_geojson_methods import add_initial_map_column_to_gdf
from create_geojson_methods import get_list_of_flip_dict
from create_geojson_methods import implement_flip_into_new_gdf

importing Jupyter notebook from create_geojson_methods.ipynb


In [2]:
tracts_fname = 'data/tl_2013_55_tract.zip'
partition_fname = 'data/wi_gerrymander_dem.csv'
json_fname = 'data/wi_path_100flips.json'
population_fname = 'data/wi_tract_populations_census_2010.csv'
voteshares_fname = 'data/wi_voteshares.csv'

metric_fn_names = ['compute_SL_index', 'compute_efficiency_gap', 'compute_mm_gap']
metric_gdf_column_names = ['SL_index', 'efficiency_gap', 'mm_gap']

In [3]:
def set_up_original_gdf(map_as_gdf):
    # cleans gdf and sets up initial district column
    map_as_gdf = clean_gdf_columns(map_as_gdf)
    map_data_dict = get_initial_map_data(json_fname)
    map_as_gdf = add_initial_map_column_to_gdf(map_data_dict, map_as_gdf)
    
    # adds population and voteshare data
    map_as_gdf = helpers.add_population_data(map_as_gdf, population_fname)
    map_as_gdf = helpers.add_voteshare_data(map_as_gdf, voteshares_fname)
    
    return map_as_gdf

In [4]:
def create_partition(map_as_gdf):
    map_gdf_copy = map_as_gdf.copy()
    """
    Creates gerrychain parition with cleaned map_as_gdf
    """
    assignment_dict = {node: map_as_gdf['district'][node] for node in map_as_gdf.index}

    key_list = list(assignment_dict.keys())
    for node in key_list:
        if assignment_dict[node] < 0:
            assignment_dict.pop(node, None)

    map_gdf_copy.pop('district')

    map_as_partition = helpers.build_partition(map_gdf_copy, None, assignment_dict)

    return map_as_partition

In [5]:
def compute_metrics_into_gdf(map_as_partition, new_aggregate_map):
    """
    compute metrics on partition and saves to new dissolved gdf
    """
    for x in range(len(metric_fn_names)):
        metric_fn = getattr(helpers, metric_fn_names[x])
        column_name = metric_gdf_column_names[x]  
        metric_val = metric_fn(map_as_partition)

        # adds metrics computed above to new column specifying given metric
        new_aggregate_map[column_name] = metric_val
    
    return new_aggregate_map

In [6]:
if __name__ == '__main__':    
    district_counter = 2
    # sets up original gdf
    map_as_gdf = create_and_get_gdf(tracts_fname, partition_fname)
    map_as_gdf = set_up_original_gdf(map_as_gdf)
    aggregate_map = map_as_gdf.dissolve(by='district', aggfunc='sum')
    
    # creates first geojson with metrics from first district plan
    map_as_partition = create_partition(map_as_gdf)
    aggregate_map = compute_metrics_into_gdf(map_as_partition, aggregate_map)
    aggregate_map.to_file("geojson/wi_map_plan_1.geojson", driver='GeoJSON')    
        
    flip_json_list_of_dict = get_list_of_flip_dict(json_fname)
    
    for one_flip_dict in flip_json_list_of_dict:
        new_map_as_gdf = implement_flip_into_new_gdf(one_flip_dict, map_as_gdf)
        
        new_aggregate_map = new_map_as_gdf.dissolve(by='district', aggfunc='sum')
        
        # creates gerrychain partition object
        new_map_as_partition = create_partition(new_map_as_gdf)
        
        new_aggregate_map = compute_metrics_into_gdf(new_map_as_partition, new_aggregate_map)
        
        # saves gdf with metric columns to geojson
        new_aggregate_map.to_file("geojson/wi_map_plan_" + str(district_counter) + ".geojson", driver='GeoJSON')
        
        district_counter = district_counter + 1

/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/gerrychain/graph/graph.py:152: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


	GOP vote shares: [0.5478 0.5335 0.3439 0.342  0.5387 0.5067 0.5366 0.5255]


/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/gerrychain/graph/graph.py:152: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


	GOP vote shares: [0.5474 0.5335 0.3439 0.342  0.5387 0.5067 0.5366 0.5257]


/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/gerrychain/graph/graph.py:152: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


...

	GOP vote shares: [0.548  0.5335 0.3439 0.342  0.5385 0.5067 0.5366 0.5255]


/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/gerrychain/graph/graph.py:152: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


	GOP vote shares: [0.5478 0.5335 0.3439 0.3418 0.5387 0.5079 0.5366 0.5255]


/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/gerrychain/graph/graph.py:152: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


	GOP vote shares: [0.5478 0.5335 0.3439 0.3399 0.5387 0.5078 0.5366 0.5255]
